In [107]:
import requests
import keys
import pandas as pd

In [108]:
# def fetch(fields, max_per_call=500):
#     base_url = 'https://api.igdb.com/v4/age_ratings'
#     headers = {
#         'Client-ID': keys.API_CLIENT_ID,  
#         'Authorization': f'Bearer {keys.API_AUTHORIZATION_TOKEN}', 
#     }
    
#     all_data = []
#     offset = 0
    
#     while True:
#         data = f"fields {','.join(fields)}; limit {max_per_call}; offset {offset};"
#         response = requests.post(base_url, headers=headers, data=data)
        
#         if response.status_code != 200:
#             print(f"Error: {response.status_code}, {response.text}")
#             break
        
#         games = response.json()  
        
#         if not games:
#             break
#         all_data.extend(games)  
#         print(f"Scraped {offset+500} games")
#         if len(games) < max_per_call:
#             break
        
#         offset += max_per_call  
    
#     return all_data  


In [109]:
# fetched_games = fetch(fields)


In [110]:
# print(len(fetched_games))
# print(fetched_games[10])

In [111]:
import requests

def fetch():
    base_url = 'https://api.igdb.com/v4/games'
    headers = {
        'Client-ID': keys.API_CLIENT_ID,  
        'Authorization': f'Bearer {keys.API_AUTHORIZATION_TOKEN}', 
    }
    
    # Define the query for the game
    data = """fields 
        id, name, slug, summary, url, total_rating,
        age_ratings.rating, age_ratings.category,
        game_modes.slug, genres.slug,
        involved_companies.company.slug, involved_companies.developer, involved_companies.publisher,
        keywords.slug, platforms.slug, player_perspectives.slug,
        release_dates.date, release_dates.platform.slug,
        artworks.url, artworks.image_id,
        cover.url, cover.image_id,
        screenshots.image_id, screenshots.url,
        similar_games.slug,
        videos.name, videos.video_id,
        websites.url;
        where id = 75235;
    """
    
    try:
        response = requests.post(base_url, headers=headers, data=data)
        
        if response.status_code != 200:
            print(f"Error: {response.status_code}, {response.text}")
            return
        
        games = response.json()  
        
        if not games:
            print("No game data found.")
            return None

        return games
    
    except requests.RequestException as e:
        print(f"Request failed: {e}")
        return None


In [112]:
fetched=fetch()

In [113]:
df=pd.DataFrame(fetched)

In [114]:
data=df[['age_ratings','game_modes','genres','involved_companies','keywords','platforms','player_perspectives','release_dates','total_rating']]
df.head()

,id,age_ratings,artworks,cover,game_modes,genres,involved_companies,keywords,name,platforms,player_perspectives,release_dates,screenshots,similar_games,slug,summary,total_rating,url,videos,websites
0,75235,"[{'id': 32742, 'category': 2, 'rating': 5}, {'...","[{'id': 5086, 'image_id': 'j7dbzjr8ddshnaewmb4...","{'id': 109855, 'image_id': 'co2crj', 'url': '/...","[{'id': 1, 'slug': 'single-player'}]","[{'id': 12, 'slug': 'role-playing-rpg'}, {'id'...","[{'id': 56819, 'company': {'id': 403, 'slug': ...","[{'id': 22, 'slug': 'assassin'}, {'id': 253, '...",Ghost of Tsushima,"[{'id': 48, 'slug': 'ps4--1'}]","[{'id': 2, 'slug': 'third-person'}]","[{'id': 223511, 'date': 1594944000, 'platform'...","[{'id': 132328, 'image_id': 'hcfpcjiout0cmtttf...","[{'id': 10776, 'slug': 'rising-world'}, {'id':...",ghost-of-tsushima,Uncover the hidden wonders of Tsushima in this...,90.473244,https://www.igdb.com/games/ghost-of-tsushima,"[{'id': 16583, 'name': 'PGW17 Trailer', 'video...","[{'id': 60919, 'url': 'https://www.suckerpunch..."


In [115]:
print(data['age_ratings'])

0    [{'id': 32742, 'category': 2, 'rating': 5}, {'...
Name: age_ratings, dtype: object


In [116]:
category_map = {
    1: 'ESRB', 
    2: 'PEGI', 
    3: 'CERO', 
    4: 'USK', 
    5: 'GRAC', 
    6: 'CLASS_IND', 
    7: 'ACB'
}

rating_map = {
    1: 'Three', 2: 'Seven', 3: 'Twelve', 4: 'Sixteen', 5: 'Eighteen',
    6: 'RP', 7: 'EC', 8: 'E', 9: 'E10', 10: 'T', 11: 'M', 12: 'AO',
    13: 'CERO_A', 14: 'CERO_B', 15: 'CERO_C', 16: 'CERO_D', 17: 'CERO_Z',
    18: 'USK_0', 19: 'USK_6', 20: 'USK_12', 21: 'USK_16', 22: 'USK_18',
    23: 'GRAC_ALL', 24: 'GRAC_Twelve', 25: 'GRAC_Fifteen', 26: 'GRAC_Eighteen',
    27: 'GRAC_TESTING', 28: 'CLASS_IND_L', 29: 'CLASS_IND_Ten', 30: 'CLASS_IND_Twelve',
    31: 'CLASS_IND_Fourteen', 32: 'CLASS_IND_Sixteen', 33: 'CLASS_IND_Eighteen',
    34: 'ACB_G', 35: 'ACB_PG', 36: 'ACB_M', 37: 'ACB_MA15', 38: 'ACB_R18', 39: 'ACB_RC'
}

In [117]:
def transform_age_ratings(age_ratings):
    return [
        rating_map[rating['rating']]
        for rating in age_ratings
        if rating['category'] in [1, 2]
    ]

In [118]:
data['age_ratings'] = data['age_ratings'].apply(transform_age_ratings)

C:\Users\parth\AppData\Local\Temp\ipykernel_13148\239051527.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['age_ratings'] = data['age_ratings'].apply(transform_age_ratings)


In [119]:
def transform_game_modes(game_modes):
    return [
        modes['slug']
        for modes in game_modes
        
    ]

In [120]:
data['game_modes']=data['game_modes'].apply(transform_game_modes)

C:\Users\parth\AppData\Local\Temp\ipykernel_13148\3815974900.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['game_modes']=data['game_modes'].apply(transform_game_modes)


In [121]:
data['genres']

0    [{'id': 12, 'slug': 'role-playing-rpg'}, {'id'...
Name: genres, dtype: object

In [122]:
def transform_genres(genres):
    return [
        genre['slug']
        for genre in genres
        
    ]

In [123]:
data['genres']=df['genres'].apply(transform_genres)

C:\Users\parth\AppData\Local\Temp\ipykernel_13148\600558847.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['genres']=df['genres'].apply(transform_genres)


In [124]:
data.head()

,age_ratings,game_modes,genres,involved_companies,keywords,platforms,player_perspectives,release_dates,total_rating
0,"[Eighteen, M]",[single-player],"[role-playing-rpg, hack-and-slash-beat-em-up, ...","[{'id': 56819, 'company': {'id': 403, 'slug': ...","[{'id': 22, 'slug': 'assassin'}, {'id': 253, '...","[{'id': 48, 'slug': 'ps4--1'}]","[{'id': 2, 'slug': 'third-person'}]","[{'id': 223511, 'date': 1594944000, 'platform'...",90.473244


In [125]:
data['involved_companies']

0    [{'id': 56819, 'company': {'id': 403, 'slug': ...
Name: involved_companies, dtype: object